In [ ]:
!nvidia-smi
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:
!pip install open_clip_torch matplotlib

In [17]:
import numpy as np
import torch

In [19]:
import open_clip
open_clip.list_pretrained()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
!git clone https://github.com/crowsonkb/cloob-training
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
%cd /content/cloob-training/
!pip install -r requirements.txt
from cloob_training import model_pt, pretrained

pretrained.list_configs()

In [ ]:
config = pretrained.get_config('cloob_laion_400m_vit_b_16_32_epochs')
model = model_pt.get_pt_model(config)

checkpoint = pretrained.download_checkpoint(config)
model.load_state_dict(model_pt.get_pt_params(config, checkpoint))

model.eval().requires_grad_(False)#.to('cuda')
model.normalize

In [24]:
image_size = (224,224)

def _convert_to_rgb(image):
    return image.convert('RGB')

t_form = Compose([
            Resize(image_size, interpolation=InterpolationMode.BICUBIC),
            CenterCrop(image_size),
            _convert_to_rgb,
            ToTensor()])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [26]:
import pandas as pd

#Define text stimuli
female = ['female', 'woman', 'girl', 'sister', 'she', 'her', 'hers', 'daughter']
male = ['male', 'man', 'boy', 'brother', 'he', 'him', 'his', 'son']
all_words = female + male

embeddings = []

for word in all_words:
  with torch.no_grad():
    tokens = model.tokenize([word])
    emb = model.text_encoder(tokens).numpy().squeeze()
    embeddings.append(emb)

emb_arr = np.array(embeddings)
emb_df = pd.DataFrame(emb_arr,index=all_words)
emb_df.to_csv(f'/content/drive/My Drive/cloob/clip_cloob_emb_df_language.vec',sep=' ')

In [ ]:
from PIL import Image
from os import listdir
from torchvision import transforms, datasets, models
from torchvision.transforms import Normalize, Compose, RandomResizedCrop, InterpolationMode, ToTensor, Resize, \
    CenterCrop

SOURCE_DIR = f'/content/drive/My Drive/OASIS/images/'

targets = listdir(SOURCE_DIR)
embeddings = []

for target in targets:

  print(target)

  img = Image.open(f'{SOURCE_DIR}{target}')
  img_ = t_form(img)
  input_ = model.normalize(img_).unsqueeze(0)
  emb = model.image_encoder(input_).numpy().squeeze()
  embeddings.append(emb)

emb_arr = np.array(embeddings)
emb_df = pd.DataFrame(emb_arr,index=targets)
emb_df.to_csv(f'/content/drive/My Drive/cloob/clip_cloob_emb_df.vec',sep=' ')

In [28]:
#Define text stimuli
person = ['person','woman','human','human being','individual','adult']
happy = ['happy person','happy woman','happy human','happy human being','happy individual','happy adult']
sad = ['sad person','sad woman','sad human','sad human being','sad individual','sad adult']
angry = ['angry person','angry woman','angry human','angry human being','angry individual','angry adult']

person_ = person + [f'a {i}' for i in person] + [f'a photo of a {i}' for i in person] + [f'an image of a {i}' for i in person] + [f'a picture of a {i}' for i in person]
happy_ = happy + [f'a {i}' for i in happy] + [f'a photo of a {i}' for i in happy] + [f'an image of a {i}' for i in happy] + [f'a picture of a {i}' for i in happy]
sad_ = sad + [f'a {i}' for i in sad] + [f'a photo of a {i}' for i in sad] + [f'an image of a {i}' for i in sad] + [f'a picture of a {i}' for i in sad]
angry_ = angry + [f'an {i}' for i in angry] + [f'a photo of an {i}' for i in angry] + [f'an image of an {i}' for i in angry] + [f'a picture of an {i}' for i in angry]

all_words = person_ + happy_ + sad_ + angry_

embeddings = []

for word in all_words:
  with torch.no_grad():
    tokens = model.tokenize([word])
    emb = model.text_encoder(tokens).numpy().squeeze()
    embeddings.append(emb)

emb_arr = np.array(embeddings)
emb_df = pd.DataFrame(emb_arr,index=all_words)
emb_df.to_csv(f'/content/drive/My Drive/cloob/sobem_clip_cloob_emb_lang_df.vec',sep=' ')

In [ ]:
SOURCE_DIR = f'/content/drive/My Drive/sobem/Photos/'

embeddings, df_index = [],[]

for i in range(1,11):
  target_dir = f'{SOURCE_DIR}{str(i)}/'
  targets = listdir(target_dir)

  for target in targets:

    print(target)

    img = Image.open(f'{target_dir}{target}')
    img_ = t_form(img)
    input_ = model.normalize(img_).unsqueeze(0)
    emb = model.image_encoder(input_).numpy().squeeze()
    embeddings.append(emb)
    df_index.append(target)

emb_arr = np.array(embeddings)
emb_df = pd.DataFrame(emb_arr,index=df_index)
emb_df.to_csv(f'/content/drive/My Drive/cloob/sobem_clip_cloob_emb_df.vec',sep=' ')

In [31]:
#Define text stimuli
sex = ['person to have intercourse with','person to be intimate with','person to have sex with','person to kiss','person to undress','person to have coitus with']
sex_person = sex + [f'a {i}' for i in sex] + [f'a photo of a {i}' for i in sex] + [f'an image of a {i}' for i in sex] + [f'a picture of a {i}' for i in sex]

science = ['scientist','researcher','engineer','physicist','mathematician','chemist']
sci_person = science + [f'a {i}' for i in science] + [f'a photo of a {i}' for i in science] + [f'an image of a {i}' for i in science] + [f'a picture of a {i}' for i in science]

business = ['businessperson', 'business leader', 'manager', 'executive', 'CEO', 'chief executive officer']
bus_person = business + [f'a {i}' for i in business] + [f'a photo of a {i}' for i in business] + [f'an image of a {i}' for i in business] + [f'a picture of a {i}' for i in business]

medicine = ['doctor', 'physician', 'clinician','surgeon', 'medical expert', 'health professional']
med_person = medicine + [f'a {i}' for i in medicine] + [f'a photo of a {i}' for i in medicine] + [f'an image of a {i}' for i in medicine] + [f'a picture of a {i}' for i in medicine]

all_words = sex_person + sci_person + bus_person + med_person

embeddings = []

for word in all_words:
  with torch.no_grad():
    tokens = model.tokenize([word])
    emb = model.text_encoder(tokens).numpy().squeeze()
    embeddings.append(emb)

emb_arr = np.array(embeddings)
emb_df = pd.DataFrame(emb_arr,index=all_words)
emb_df.to_csv(f'/content/drive/My Drive/cloob/profession_clip_cloob_emb_lang_df.vec',sep=' ')

In [35]:
SOURCE_DIR = f'/content/drive/My Drive/profession_stimuli/'
df_index = []
image_embeddings = []

professions = ['ceo','doctor','scientist']
genders = ['men','women']

for profession in professions:
    for gender in genders:
        target_dir = f'{SOURCE_DIR}{profession}/{gender}/'
        imgs = listdir(target_dir)

        for idx,img in enumerate(imgs):
          image = Image.open(f'{target_dir}{img}')
          img_ = t_form(image)
          input_ = model.normalize(img_).unsqueeze(0)
          with torch.no_grad():
            emb = model.image_encoder(input_).numpy().squeeze()
          image_embeddings.append(emb)
          df_index.append(f'{profession}_{gender}_{img}')

image_array = np.array(image_embeddings)
image_df = pd.DataFrame(image_array,index=df_index)
image_df.to_csv(f'/content/drive/My Drive/cloob/profession_clip_cloob_emb_df.vec',sep=' ')